In [1]:
# Install necessary Python pakcages
# rdflib: RDF composition and querying with SPARQL in Python.
from rdflib import RDFS, RDF, Namespace, Graph, URIRef, Literal

from common import *

In [2]:
import json
import pandas as pd
import re

In [3]:
# Load knowledge for the conversion.
with open('metadata/re_dict.json', 'r') as fp:
    re_dict = json.load(fp)
equip_order = ['AHU', 'Return_Fan', 'Supply_Fan']
with open('metadata/zone_room_map.json', 'r') as fp:
    zone_room_map = json.load(fp)
    
# Load raw data
df = pd.read_csv('metadata/brick_sample_building_raw.csv')

In [4]:
# Consruct knowledge from Brick schema

bg = Graph()
bg.parse('Brick/dist/Brick.ttl', format='turtle') # Load Brick schema.
subclasses_query = """
PREFIX brick: <https://brickschema.org/schema/1.0.1/Brick#>
select ?tagset where {{
?tagset rdfs:subClassOf+ brick:{0}.
}}
"""
point_query = subclasses_query.format('Point')
equip_query = subclasses_query.format('Equipment')
loc_query = subclasses_query.format('Location')
points = [str(row[0]).split('#')[-1] for row in bg.query(point_query)]
equips = [str(row[0]).split('#')[-1] for row in bg.query(equip_query)]
locs = [str(row[0]).split('#')[-1] for row in bg.query(loc_query)]

# Cleanup rules
equips = [equip for equip in equips if \
          not 'Command' in equip and
          not 'Sensor' in equip and
          not 'Status' in equip and
          not 'Alarm' in equip
         ]

In [5]:
g = Graph() # Initialize a graph
RDFS # predefined namespace as 'http://www.w3.org/2000/01/rdf-schema#'
RDF # predefined namespace as 'http://www.w3.org/1999/02/22-rdf-syntax-ns#'
BRICK = Namespace('https://brickschema.org/schema/1.0.1/Brick#')
BF = Namespace('https://brickschema.org/schema/1.0.1/BrickFrame#')
EX = Namespace('http://example.com#')
g.bind('ex', EX)
g.bind('brick', BRICK)
g.bind('bf', BF)
g.bind('rdfs', RDFS)
g.bind('rdf', RDF)



In [6]:
for row in df.iterrows():
    vendor_name = row[1]['Vendor Given Name']
    uuid = row[1]['uuid']
    ############# Parsing Phase
    # Extract all entities in vendor_name
    entity_dict = dict()
    raw_words = vendor_name.split('.') # This building uses '.' as a delimiter but not always it's comprehensive.
    
    # split more if needed
    words = []
    for word in raw_words:
        if 'SF' in word or 'RF' in word:
            words += word.split('-')
        else:
            words.append(word)
    # Apply RE rules to detect tagsets.
    for word in words:
        for key, tagset in re_dict.items():
            if re.findall(key, word):
                if tagset in points:
                    entity_dict[vendor_name] = tagset
                else:
                    entity_dict[word] = tagset # We will use a word as an entity name.
    
    ############# Add Brick Relationships
    # Add instance relationships
    for entity, tagset in entity_dict.items():
        g.add((EX[entity], RDF['type'], BRICK[tagset]))
    
    # Add Location <-> Others
    ## Find all location entities
    loc_entities = list()
    for entity, tagset in entity_dict.items():
        if tagset in locs:
            loc_entities.append(entity)
    ## Assign hasLocation relationships to all non-location entities with the location entities.
    for entity, tagset in entity_dict.items():
        if tagset not in locs:
            for loc_entity in loc_entities:
                g.add((EX[entity], BF['hasLocation'], EX[loc_entity]))
    
    # Zone corresponds to a VAV.
    vav_entity = None
    for entity, tagset in entity_dict.items():
        if tagset == 'HVAC_Zone':
            vav_entity = 'VAV_' + entity
            g.add((EX[vav_entity], RDF['type'], BRICK['VAV']))
            g.add((EX[vav_entity], BF['feeds'], EX[entity]))
    if vav_entity:
        entity_dict[vav_entity] = 'VAV'
                
    # Add Equip <-> Point
    ## Find all equip entities
    equip_entities = list()
    for entity, tagset in entity_dict.items():
        if tagset in equips:
            equip_entities.append(entity)
    ## Assign isPointOf relationships to all Point entities 
    for entity, tagset in entity_dict.items():
        if tagset in points:
            for equip_entity in equip_entities:
                g.add((EX[entity], BF['isPointOf'], EX[equip_entity]))
    ## Add inclusive relationships among equipments
    #if len(equip_entities) > 1:
    #    rank_equip_list = [(equip, equip_orders.index(equip))]
    #    sorted(equip_entities)
    
    # Add UUID
    for entity, tagset in entity_dict.items():
        if tagset in points:
            g.add((EX[entity], BRICK['hasUuid'], Literal(str(uuid))))

In [7]:
# Add zone-room inclusive relationships
for zone, rooms in zone_room_map.items():
    for room in rooms:
        g.add((EX[room], RDF['type'], BRICK['Room']))
        g.add((EX[room], BF['isPartOf'], EX[zone]))
        
# The AHU feeds all VAVs
## find VAVs.
res = g.query("select ?vav where {?vav a <https://brickschema.org/schema/1.0.1/Brick#VAV>}")
vavs = [str(row[0]).split('#')[-1] for row in res]
## Add feeds relationship
for vav in vavs:
    g.add((EX['AH1'], BF['feeds'], EX[vav]))

In [8]:
print_graph(g)

```turtle
@prefix bf: <https://brickschema.org/schema/1.0.1/BrickFrame#> .
@prefix brick: <https://brickschema.org/schema/1.0.1/Brick#> .
@prefix ex: <http://example.com#> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix xml: <http://www.w3.org/XML/1998/namespace> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

ex:AH1-1 a brick:AHU ;
    bf:hasLocation ex:BLDA .

ex:BLDA.AH1.CHWR1-T a brick:Chilled_Water_Return_Temperature_Sensor ;
    brick:hasUuid "1" ;
    bf:hasLocation ex:BLDA ;
    bf:isPointOf ex:AH1 .

ex:BLDA.AH1.CHWR2-T a brick:Chilled_Water_Return_Temperature_Sensor ;
    brick:hasUuid "2" ;
    bf:hasLocation ex:BLDA ;
    bf:isPointOf ex:AH1 .

ex:BLDA.AH1.CLG-VLV1 a brick:Cooling_Valve_Command ;
    brick:hasUuid "3" ;
    bf:hasLocation ex:BLDA ;
    bf:isPointOf ex:AH1 .

ex:BLDA.AH1.CLG-VLV2 a brick:Cooling_Valve_Command ;
    brick:hasUuid "4" ;
    bf:hasLocation ex:BLDA ;
    bf:isPointOf ex:AH1 .

ex:BLDA.RM-1100A..ACTCLGSP a brick:Effective_Cooling_Temperature_Setpoint ;
    brick:hasUuid "15" ;
    bf:hasLocation ex:BLDA,
        ex:RM-1100A ;
    bf:isPointOf ex:VAV_RM-1100A .

ex:BLDA.RM-1100A..ACTHTGSP a brick:Effective_Heating_Temperature_Setpoint ;
    brick:hasUuid "16" ;
    bf:hasLocation ex:BLDA,
        ex:RM-1100A ;
    bf:isPointOf ex:VAV_RM-1100A .

ex:BLDA.RM-1100A..COMMONSP a brick:Zone_Temperature_Setpoint ;
    brick:hasUuid "18" ;
    bf:hasLocation ex:BLDA,
        ex:RM-1100A ;
    bf:isPointOf ex:VAV_RM-1100A .

ex:BLDA.RM-1100A..DPR-C a brick:Damper_Command ;
    brick:hasUuid "20" ;
    bf:hasLocation ex:BLDA,
        ex:RM-1100A ;
    bf:isPointOf ex:VAV_RM-1100A .

ex:BLDA.RM-1100A..OCC-CMD a brick:Occupied_Command ;
    brick:hasUuid "22" ;
    bf:hasLocation ex:BLDA,
        ex:RM-1100A ;
    bf:isPointOf ex:VAV_RM-1100A .

ex:BLDA.RM-1100A..RH-VLV a brick:Reheat_Valve_Command ;
    brick:hasUuid "23" ;
    bf:hasLocation ex:BLDA,
        ex:RM-1100A ;
    bf:isPointOf ex:VAV_RM-1100A .

ex:BLDA.RM-1100A..SUP-FLOW a brick:Supply_Air_Flow_Sensor ;
    brick:hasUuid "25" ;
    bf:hasLocation ex:BLDA,
        ex:RM-1100A ;
    bf:isPointOf ex:VAV_RM-1100A .

ex:BLDA.RM-1100A..SUPFLO-SP a brick:Supply_Air_Flow_Setpoint ;
    brick:hasUuid "24" ;
    bf:hasLocation ex:BLDA,
        ex:RM-1100A ;
    bf:isPointOf ex:VAV_RM-1100A .

ex:BLDA.RM-1100D..ACTCLGSP a brick:Effective_Cooling_Temperature_Setpoint ;
    brick:hasUuid "27" ;
    bf:hasLocation ex:BLDA,
        ex:RM-1100D ;
    bf:isPointOf ex:VAV_RM-1100D .

ex:BLDA.RM-1100D..ACTHTGSP a brick:Effective_Heating_Temperature_Setpoint ;
    brick:hasUuid "28" ;
    bf:hasLocation ex:BLDA,
        ex:RM-1100D ;
    bf:isPointOf ex:VAV_RM-1100D .

ex:BLDA.RM-1100D..COMMONSP a brick:Zone_Temperature_Setpoint ;
    brick:hasUuid "30" ;
    bf:hasLocation ex:BLDA,
        ex:RM-1100D ;
    bf:isPointOf ex:VAV_RM-1100D .

ex:BLDA.RM-1100D..DPR-C a brick:Damper_Command ;
    brick:hasUuid "32" ;
    bf:hasLocation ex:BLDA,
        ex:RM-1100D ;
    bf:isPointOf ex:VAV_RM-1100D .

ex:BLDA.RM-1100D..OCC-CMD a brick:Occupied_Command ;
    brick:hasUuid "34" ;
    bf:hasLocation ex:BLDA,
        ex:RM-1100D ;
    bf:isPointOf ex:VAV_RM-1100D .

ex:BLDA.RM-1100D..RH-VLV a brick:Reheat_Valve_Command ;
    brick:hasUuid "35" ;
    bf:hasLocation ex:BLDA,
        ex:RM-1100D ;
    bf:isPointOf ex:VAV_RM-1100D .

ex:BLDA.RM-1100D..SUP-FLOW a brick:Supply_Air_Flow_Sensor ;
    brick:hasUuid "37" ;
    bf:hasLocation ex:BLDA,
        ex:RM-1100D ;
    bf:isPointOf ex:VAV_RM-1100D .

ex:BLDA.RM-1100D..SUPFLO-SP a brick:Supply_Air_Flow_Setpoint ;
    brick:hasUuid "36" ;
    bf:hasLocation ex:BLDA,
        ex:RM-1100D ;
    bf:isPointOf ex:VAV_RM-1100D .

ex:BLDA.RM-2150..ACTCLGSP a brick:Effective_Cooling_Temperature_Setpoint ;
    brick:hasUuid "39" ;
    bf:hasLocation ex:BLDA,
        ex:RM-2150 ;
    bf:isPointOf ex:VAV_RM-2150 .

ex:BLDA.RM-2150..ACTHTGSP a brick:Effective_Heating_Temperature_Setpoint ;
    brick:hasUuid "40" ;
    bf:hasLocation ex:BLDA,
        ex:RM-2150 ;
    bf:isPointOf ex:VAV_RM-2150 .

ex:BLDA.RM-2150..COMMONSP a brick:Zone_Temperature_Setpoint ;
    brick:hasUuid "42" ;
    bf:hasLocation ex:BLDA,
        ex:RM-2150 ;
    bf:isPointOf ex:VAV_RM-2150 .

ex:BLDA.RM-2150..DPR-C a brick:Damper_Command ;
    brick:hasUuid "44" ;
    bf:hasLocation ex:BLDA,
        ex:RM-2150 ;
    bf:isPointOf ex:VAV_RM-2150 .

ex:BLDA.RM-2150..HTGFLOW a brick:Occupied_Heating_Discharge_Air_Flow_Setpoint ;
    brick:hasUuid "45" ;
    bf:hasLocation ex:BLDA,
        ex:RM-2150 ;
    bf:isPointOf ex:VAV_RM-2150 .

ex:BLDA.RM-2150..OCC-CMD a brick:Occupied_Command ;
    brick:hasUuid "47" ;
    bf:hasLocation ex:BLDA,
        ex:RM-2150 ;
    bf:isPointOf ex:VAV_RM-2150 .

ex:BLDA.RM-2150..RH-VLV a brick:Reheat_Valve_Command ;
    brick:hasUuid "48" ;
    bf:hasLocation ex:BLDA,
        ex:RM-2150 ;
    bf:isPointOf ex:VAV_RM-2150 .

ex:BLDA.RM-2150..SUP-FLOW a brick:Supply_Air_Flow_Sensor ;
    brick:hasUuid "50" ;
    bf:hasLocation ex:BLDA,
        ex:RM-2150 ;
    bf:isPointOf ex:VAV_RM-2150 .

ex:BLDA.RM-2150..SUPFLO-SP a brick:Supply_Air_Flow_Setpoint ;
    brick:hasUuid "49" ;
    bf:hasLocation ex:BLDA,
        ex:RM-2150 ;
    bf:isPointOf ex:VAV_RM-2150 .

ex:RF a brick:Return_Fan ;
    bf:hasLocation ex:BLDA .

ex:RF1 a brick:Return_Fan ;
    bf:hasLocation ex:BLDA .

ex:RF2 a brick:Return_Fan ;
    bf:hasLocation ex:BLDA .

ex:RM-1100 a brick:Room ;
    bf:isPartOf ex:RM-1100A,
        ex:RM-1100D .

ex:RM-2151 a brick:Room ;
    bf:isPartOf ex:RM-2150 .

ex:RM-2152 a brick:Room ;
    bf:isPartOf ex:RM-2150 .

ex:SF a brick:Supply_Fan ;
    bf:hasLocation ex:BLDA .

ex:SF1 a brick:Supply_Fan ;
    bf:hasLocation ex:BLDA .

ex:SF2 a brick:Supply_Fan ;
    bf:hasLocation ex:BLDA .

ex:AH1 a brick:AHU ;
    bf:feeds ex:VAV_RM-1100A,
        ex:VAV_RM-1100D,
        ex:VAV_RM-2150 ;
    bf:hasLocation ex:BLDA .

ex:VAV_RM-1100A a brick:VAV ;
    bf:feeds ex:RM-1100A .

ex:VAV_RM-1100D a brick:VAV ;
    bf:feeds ex:RM-1100D .

ex:RM-1100A a brick:HVAC_Zone .

ex:RM-1100D a brick:HVAC_Zone .

ex:VAV_RM-2150 a brick:VAV ;
    bf:feeds ex:RM-2150 .

ex:RM-2150 a brick:HVAC_Zone .

ex:BLDA a brick:Building .


```

### Conversion Rules (for this building)
1. A metadata is the name of the data point.
2. Implicit Relationships:
  - When a location is found, every other entitiy hasLocation the location.
  - When equipment is is found, the point isPointOf the equipment.
  - When multiple equipment is found, their hierarchy is assumed. In AHU-SF, SF isPartOf AHU. Same for AHU-RF.
  - When HVAC Zone is found, there is a corresponding VAV.

In [9]:
g.serialize('metadata/sample_building.ttl', format='turtle')